In [15]:
from numpy import vstack, argmax
from pandas import read_csv
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from torch import Tensor
from torch.utils.data import Dataset, DataLoader, random_split
from torch.nn import Linear, ReLU, Softmax, Module, Dropout
from torch.optim import Adam
from torch.nn import CrossEntropyLoss
from torch.nn.init import kaiming_uniform_, xavier_uniform_
from tqdm import tqdm
import numpy as np
import torch
import plotly.graph_objects as go


In [16]:
class CSVDataset(Dataset):
    def __init__(self, path):
        df = read_csv(path, header=0)

        # Exclude non-numeric columns and the 'UID' column
        non_numeric_columns = ['Product ID', 'Type', 'Target', 'Failure Type']
        df_numeric = df.drop(columns=non_numeric_columns)
        self.X = df_numeric.values.astype('float32')

        # Encode non-numeric categorical columns
        label_encoders = {}
        for column in non_numeric_columns:
            label_encoders[column] = LabelEncoder()
            df[column] = label_encoders[column].fit_transform(df[column])

        # Assign the encoded 'Target' column to y
        self.y = df['Target'].values

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return [self.X[idx], self.y[idx]]

    def get_splits(self, n_test=0.33):
        test_size = round(n_test * len(self.X))
        train_size = len(self.X) - test_size
        return random_split(self, [train_size, test_size])


In [17]:
#model definition 
class MLP(Module):
    def __init__(self, n_inputs):
        super(MLP, self).__init__()
        self.hidden1 = Linear(n_inputs, 10)
        kaiming_uniform_(self.hidden1.weight, nonlinearity='relu')
        self.act1 = ReLU()
        self.dropout1 = Dropout(0.2)
        self.hidden2 = Linear(10, 8)
        kaiming_uniform_(self.hidden2.weight, nonlinearity='relu')
        self.act2 = ReLU()
        self.hidden3 = Linear(8, 3)
        xavier_uniform_(self.hidden3.weight)
        self.act3 = Softmax(dim=1)

    def forward(self, X):
        X = self.hidden1(X)
        X = self.act1(X)
        X = self.dropout1(X)
        X = self.hidden2(X)
        X = self.act2(X)
        X = self.hidden3(X)
        X = self.act3(X)
        return X

In [18]:
def prepare_data(path):
    dataset = CSVDataset(path)
    train, test = dataset.get_splits()
    train_dl = DataLoader(train, batch_size=1024, shuffle=True)
    test_dl = DataLoader(test, batch_size=1024, shuffle=False)
    return train_dl, test_dl

In [19]:
class EarlyStopping:
    def __init__(self, patience=7, verbose=False, delta=0, path='checkpoint.pt', trace_func=print):
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func

    def __call__(self, val_loss, model):
        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        if self.verbose:
            self.trace_func(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

number_epochs = 500
learning_rate = 0.01
loss_per_epoch = []
loss_per_epoch_validation = []

In [20]:
def train_model(train_dl, model):
    patience = 7
    optimizer =  Adam(model.parameters(), lr=learning_rate , weight_decay= 0.01)
    early_stopping = EarlyStopping(patience=patience, verbose=True)
    
    for epoch in tqdm(range(number_epochs), desc='Training Epochs'):
        print(f"Epoch {epoch+1}\n-------------------------------")
        
        total_loss = 0.0
        model.train()
        TrainingStep_loss = []
        
        for batch, (inputs, targets) in enumerate(train_dl):
            optimizer.zero_grad()
            yhat = model(inputs)
            loss = CrossEntropyLoss()(yhat, targets)
            loss.backward()
            optimizer.step()
            TrainingStep_loss.append(loss.item())
            
        loss = np.array(TrainingStep_loss).mean()
        loss_per_epoch.append(loss)
        print(f"loss: {loss:>7f}")
        
        model.eval()
        ValidationStep_loss = []
        
        for batch, (inputs, targets) in enumerate(test_dl):
            outputs = model(inputs)
            validation_loss = CrossEntropyLoss()(outputs, targets)
            ValidationStep_loss.append(validation_loss.item())
        
        loss_per_epoch_validation.append(np.array(ValidationStep_loss).mean())
        
        train_loss = np.average(TrainingStep_loss)
        valid_loss = np.average(ValidationStep_loss)
        
        early_stopping(valid_loss, model)
        
        if early_stopping.early_stop:
            print("Early stopping")
            break


In [21]:
# evaluate the model
def evaluate_model(test_dl, model):
    predictions, actuals = list(), list()
    for i, (inputs, targets) in enumerate(test_dl):
        yhat = model(inputs)
        yhat = yhat.detach().numpy()
        actual = targets.numpy()
        yhat = argmax(yhat, axis=1)
        actual = actual.reshape((len(actual), 1))
        yhat = yhat.reshape((len(yhat), 1))
        predictions.append(yhat)
        actuals.append(actual)
    predictions, actuals = vstack(predictions), vstack(actuals)
    acc = accuracy_score(actuals, predictions)
    return acc

#make a class prediction for one row of data 
def predict(row, model):
    row = Tensor([row])
    yhat = model(row)
    yhat = yhat.detach().numpy()
    return yhat


In [22]:
# prepare the data

path = 'C:/Users/damia/Downloads/predictive maintenance dataset/predictive_maintenance.csv'
train_dl, test_dl = prepare_data(path)
print(len(train_dl.dataset), len(test_dl.dataset))

# define the network
model = MLP(6)  # Adjust the input size based on your dataset features

# train the model
train_model(train_dl, model)

6700 3300


Training Epochs:   0%|▏                                                                | 1/500 [00:00<01:32,  5.40it/s]

Epoch 1
-------------------------------
loss: 0.705374
Validation loss decreased (inf --> 0.581954).  Saving model ...


Training Epochs:   0%|▎                                                                | 2/500 [00:00<01:11,  6.92it/s]

Epoch 2
-------------------------------
loss: 0.593453
Validation loss decreased (0.581954 --> 0.581954).  Saving model ...
Epoch 3
-------------------------------
loss: 0.585992


...

loss: 0.585106
EarlyStopping counter: 3 out of 7
Epoch 61
-------------------------------
loss: 0.585143


Training Epochs:  12%|███████▉                                                        | 62/500 [00:08<01:00,  7.21it/s]

EarlyStopping counter: 4 out of 7
Epoch 62
-------------------------------
loss: 0.584447
EarlyStopping counter: 5 out of 7
Epoch 63
-------------------------------


Training Epochs:  13%|████████                                                        | 63/500 [00:09<01:04,  6.78it/s]

loss: 0.585991
EarlyStopping counter: 6 out of 7
Epoch 64
-------------------------------
loss: 0.585739
EarlyStopping counter: 7 out of 7
Early stopping


In [23]:
# evaluate the model
acc = evaluate_model(test_dl, model)
print('Accuracy: %.3f' % acc)

Accuracy: 0.966


In [24]:
#. loss function curve 
import plotly.graph_objects as go

# Assuming you have loss_per_epoch and loss_per_epoch_validation lists
#loss_per_epoch = [0.5, 0.4, 0.3]  # Replace with your actual list
#loss_per_epoch_validation = [0.6, 0.5, 0.4]  # Replace with your actual list

# Create the figure
fig = go.Figure()

# Add training loss trace
fig.add_trace(go.Scatter(x=list(range(len(loss_per_epoch))),
                         y=loss_per_epoch,
                         mode='lines',
                         name='train'))

# Add validation loss trace
fig.add_trace(go.Scatter(x=list(range(len(loss_per_epoch_validation))),
                         y=loss_per_epoch_validation,
                         mode='lines',
                         name='test'))

# Add labels and title
fig.update_layout(title='model loss',
                  xaxis=dict(title='epoch'),
                  yaxis=dict(title='loss'))

# Show the figure
fig.show()
#fig.write_image("LOSS_DNN_(30,20,10,1).svg")

In [25]:
# make a single prediction
row = [51,298.9,309.1,2861,4.6,143]  # Replace with your actual input features
yhat = predict(row, model)
print('Predicted: %s (class=%d)' % (yhat, argmax(yhat)))

Predicted: [[1.000000e+00 5.161137e-08 3.050285e-10]] (class=0)
